# Jiu Yang (James) Wang

## Research question/interests

My research question is looking at the correlation between housing prices and shortfall. Housing affordability is a pivotal issue for my generation, and i believe that the oft touted narrative of phantom overseas buyers is only a narrative meant to distract from the fact that the true cause of housing prices is a lack of new housing as compared to population. As such, my part of the project will be analyzing the correlation between housing prices and inventory shortfall as compared to population increases.